In [ ]:
input_root = '/home/axernox/Project_2/Lungs_Cropped/test/Sensitive'
output_root = '/home/axernox/Project_2/LIME_Output/test/Sensitive'
model_path = '/home/axernox/Project_2/efficientnetb0.keras'  
image_size = (256, 256)

model = load_model(model_path)

explainer = lime_image.LimeImageExplainer()

os.makedirs(output_root, exist_ok=True)

for root, dirs, files in os.walk(input_root):
    for file in files:
        if file.lower().endswith(('.jpg', '.jpeg', '.png')):
            img_path = os.path.join(root, file)
            
            relative_path = os.path.relpath(root, input_root)
            save_dir = os.path.join(output_root, relative_path)
            os.makedirs(save_dir, exist_ok=True)
            save_path = os.path.join(save_dir, os.path.splitext(file)[0] + '_lime.png')

            if os.path.exists(save_path):
                print(f"⏩ Skipping (already exists): {save_path}")
                continue

            try:
                img = image.load_img(img_path, target_size=image_size)
                img_array = image.img_to_array(img)

                explanation = explainer.explain_instance(
                    img_array.astype('double'),
                    model.predict,
                    top_labels=3,
                    hide_color=0,
                    num_samples=1000
                )

                temp, mask = explanation.get_image_and_mask(
                    explanation.top_labels[0],
                    positive_only=True,
                    num_features=5,
                    hide_rest=False
                )

                temp = (temp - temp.min()) / (temp.max() - temp.min())
                lime_img = mark_boundaries(temp, mask)

                lime_img = (lime_img * 255).astype(np.uint8)
                lime_pil = Image.fromarray(lime_img)
                lime_pil.save(save_path)
                print(f"✅ Saved: {save_path}")

            except Exception as e:
                print(f"❌ Error processing {img_path}: {e}")
